In [130]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import shap

In [131]:
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

In [132]:
df.describe()

C:\Users\usuario\Desktop\4geeks\Final_Proyect_Credit_Default_Risk-main\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,approval_ext,prev_apps_log,experience_score,cc_util_ext,cc_util_risk,good_behavior_score,good_behavior_ext,net_score,net_score_ext,final_score
count,252137.000000,252137.000000,252137.000000,252137.000000,2.521370e+05,252125.000000,2.518810e+05,252137.000000,252137.000000,252137.000000,...,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000
mean,278114.643103,0.086600,0.498515,172673.001234,6.113985e+05,27812.325168,5.494057e+05,0.020894,-14769.133174,-2384.169325,...,0.129144,1.475160,0.736657,0.046045,0.005233,0.095989,0.048647,0.051235,0.025861,0.568814
std,102815.635309,0.281248,0.763161,83932.565507,4.065272e+05,14647.759104,3.732685e+05,0.013874,3662.573769,2338.360162,...,0.209343,0.691497,0.379957,0.133870,0.021872,0.128917,0.072133,0.142291,0.077912,0.057929
min,100002.000000,0.000000,0.000000,25650.000000,4.500000e+04,1980.000000,4.050000e+04,0.000290,-25200.000000,-17912.000000,...,0.000000,0.000000,0.000000,-11.290221,-2.463299,0.000000,0.000000,-0.590656,-0.358986,0.294086
25%,189035.000000,0.000000,0.000000,112500.000000,2.779695e+05,17073.000000,2.475000e+05,0.010006,-17563.000000,-3175.000000,...,0.000000,1.098612,0.445402,0.000000,0.000000,0.000000,0.000000,-0.039683,-0.017823,0.530556
50%,278064.000000,0.000000,0.000000,157500.000000,5.212800e+05,25834.500000,4.500000e+05,0.018850,-14573.000000,-1648.000000,...,0.000000,1.386294,0.719086,0.000000,0.000000,0.018018,0.007733,0.000000,0.000000,0.569823
75%,367165.000000,0.000000,1.000000,211500.000000,8.292240e+05,35617.500000,6.930000e+05,0.028663,-11775.000000,-767.000000,...,0.208758,1.945910,0.992915,0.000000,0.000000,0.166667,0.077530,0.133081,0.057147,0.608071
max,456255.000000,1.000000,19.000000,472500.000000,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,0.000000,...,0.855000,4.304065,2.978956,1.999646,0.952222,0.709091,0.515572,0.709091,0.515572,0.817989


In [133]:
class_0 = df[df['TARGET'] == 0]
class_1 = df[df['TARGET'] == 1]

# Submuestrear la clase mayoritaria
clase_mayoritaria_submuestreada = resample(class_0,
                                            replace=False,
                                            n_samples=len(class_1),
                                            random_state=42)

# Combinar
df = pd.concat([clase_mayoritaria_submuestreada, class_1])

In [134]:
y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR']) #adde sk_id_curr

In [135]:
#manages divided by 0 values
X.replace([np.inf, -np.inf], np.nan, inplace=True)


In [136]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print(f"Categóricas: {len(cat_cols)}")
print(f"Numéricas: {len(num_cols)}")

Categóricas: 12
Numéricas: 313


In [137]:
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [138]:
cat_cols = X.select_dtypes(include=["category"]).columns
num_cols = X.select_dtypes(exclude=["category"]).columns

print(f"Categóricas: {len(cat_cols)}")
print(f"Numéricas: {len(num_cols)}")

Categóricas: 12
Numéricas: 313


In [139]:
num_imputer = SimpleImputer(strategy="median") #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [140]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Lista de columnas categóricas
cat_cols = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
            'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
            'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
            'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE']

# Procesar cada columna
for col in cat_cols:
    if col in df.columns:
        # 1. Rellenar NaN primero
        df[col] = df[col].fillna('Missing')

        # 2. Convertir todo a string
        df[col] = df[col].astype(str)

        # 3. Label encode
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

        # 4. IMPORTANTE: Convertir explícitamente a int
        df[col] = df[col].astype('int32')

# Verificar los tipos de datos
print(df[cat_cols].dtypes)
print("\n¿Hay valores NaN?")
print(df[cat_cols].isna().sum())

NAME_CONTRACT_TYPE            int32
CODE_GENDER                   int32
FLAG_OWN_CAR                  int32
FLAG_OWN_REALTY               int32
NAME_TYPE_SUITE               int32
NAME_INCOME_TYPE              int32
NAME_EDUCATION_TYPE           int32
NAME_FAMILY_STATUS            int32
NAME_HOUSING_TYPE             int32
OCCUPATION_TYPE               int32
WEEKDAY_APPR_PROCESS_START    int32
ORGANIZATION_TYPE             int32
dtype: object

¿Hay valores NaN?
NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
NAME_TYPE_SUITE               0
NAME_INCOME_TYPE              0
NAME_EDUCATION_TYPE           0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
OCCUPATION_TYPE               0
WEEKDAY_APPR_PROCESS_START    0
ORGANIZATION_TYPE             0
dtype: int64


In [141]:
# Ver tipos de datos de TODO el DataFrame
print(df.dtypes)

# Identificar columnas problemáticas
object_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Columnas que siguen siendo 'object': {object_cols}")

# Ver si hay NaN
print(f"\nColumnas con NaN:\n{df.isna().sum()[df.isna().sum() > 0]}")

SK_ID_CURR               int64
TARGET                   int64
NAME_CONTRACT_TYPE       int32
CODE_GENDER              int32
FLAG_OWN_CAR             int32
                        ...   
good_behavior_score    float64
good_behavior_ext      float64
net_score              float64
net_score_ext          float64
final_score            float64
Length: 327, dtype: object
Columnas que siguen siendo 'object': []

Columnas con NaN:
AMT_ANNUITY                         1
AMT_GOODS_PRICE                    38
OWN_CAR_AGE                     28322
EXT_SOURCE_1                    23688
EXT_SOURCE_2                       77
EXT_SOURCE_3                     9118
OBS_30_CNT_SOCIAL_CIRCLE          109
DEF_30_CNT_SOCIAL_CIRCLE          109
OBS_60_CNT_SOCIAL_CIRCLE          109
DEF_60_CNT_SOCIAL_CIRCLE          109
DAYS_LAST_PHONE_CHANGE              1
AMT_REQ_CREDIT_BUREAU_HOUR       6551
AMT_REQ_CREDIT_BUREAU_DAY        6551
AMT_REQ_CREDIT_BUREAU_WEEK       6551
AMT_REQ_CREDIT_BUREAU_MON        6551
AMT

In [142]:
print(X.isna().sum().sum())  # must be 0
print(X.shape)

0
(43670, 325)


In [143]:
# Ver exactamente qué columnas causan problema
for col in X.columns:
    print(f"{col}: {X[col].dtype}")
    if X[col].dtype == 'object':
        print(f"  ⚠️ COLUMNA PROBLEMÁTICA: {col}")

NAME_CONTRACT_TYPE: object
  ⚠️ COLUMNA PROBLEMÁTICA: NAME_CONTRACT_TYPE
CODE_GENDER: object
  ⚠️ COLUMNA PROBLEMÁTICA: CODE_GENDER
FLAG_OWN_CAR: object
  ⚠️ COLUMNA PROBLEMÁTICA: FLAG_OWN_CAR
FLAG_OWN_REALTY: object
  ⚠️ COLUMNA PROBLEMÁTICA: FLAG_OWN_REALTY
CNT_CHILDREN: float64
AMT_INCOME_TOTAL: float64
AMT_CREDIT: float64
AMT_ANNUITY: float64
AMT_GOODS_PRICE: float64
NAME_TYPE_SUITE: object
  ⚠️ COLUMNA PROBLEMÁTICA: NAME_TYPE_SUITE
NAME_INCOME_TYPE: object
  ⚠️ COLUMNA PROBLEMÁTICA: NAME_INCOME_TYPE
NAME_EDUCATION_TYPE: object
  ⚠️ COLUMNA PROBLEMÁTICA: NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS: object
  ⚠️ COLUMNA PROBLEMÁTICA: NAME_FAMILY_STATUS
NAME_HOUSING_TYPE: object
  ⚠️ COLUMNA PROBLEMÁTICA: NAME_HOUSING_TYPE
REGION_POPULATION_RELATIVE: float64
DAYS_BIRTH: float64
DAYS_EMPLOYED: float64
DAYS_REGISTRATION: float64
DAYS_ID_PUBLISH: float64
OWN_CAR_AGE: float64
FLAG_MOBIL: float64
FLAG_EMP_PHONE: float64
FLAG_WORK_PHONE: float64
FLAG_CONT_MOBILE: float64
FLAG_PHONE: float64
FLAG

In [144]:
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [145]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42)

In [146]:
X_train.shape


(34936, 325)

In [147]:
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
optimal_scale = neg_count / pos_count

In [148]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,
    'max_leaves': 31,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,


    'reg_alpha': 0.5,
    'reg_lambda': 2.5,
    'gamma': 1.5,
    'max_delta_step': 2,
}


xgb_model = XGBClassifier(**params, n_jobs=-1, random_state=42,  early_stopping_rounds=300, enable_categorical=True) #added enable_categorical

In [149]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.72207
[1]	validation_0-auc:0.73355
[2]	validation_0-auc:0.73679
[3]	validation_0-auc:0.73865
[4]	validation_0-auc:0.73956
[5]	validation_0-auc:0.74003
[6]	validation_0-auc:0.73990
[7]	validation_0-auc:0.74041
[8]	validation_0-auc:0.74115
[9]	validation_0-auc:0.74151
[10]	validation_0-auc:0.74147
[11]	validation_0-auc:0.74154
[12]	validation_0-auc:0.74155
[13]	validation_0-auc:0.74158
[14]	validation_0-auc:0.74158
[15]	validation_0-auc:0.74165
[16]	validation_0-auc:0.74195
[17]	validation_0-auc:0.74201
[18]	validation_0-auc:0.74205
[19]	validation_0-auc:0.74239
[20]	validation_0-auc:0.74238
[21]	validation_0-auc:0.74237
[22]	validation_0-auc:0.74262
[23]	validation_0-auc:0.74270
[24]	validation_0-auc:0.74285
[25]	validation_0-auc:0.74291
[26]	validation_0-auc:0.74293
[27]	validation_0-auc:0.74300
[28]	validation_0-auc:0.74308
[29]	validation_0-auc:0.74309
[30]	validation_0-auc:0.74296
[31]	validation_0-auc:0.74303
[32]	validation_0-auc:0.74330
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.85,
              colsample_bytree=0.75, device='cuda', early_stopping_rounds=300,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=1.5, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=2, max_depth=5,
              max_leaves=31, min_child_weight=15, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=6000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [150]:

# ==========================================
# CONFIGURACIÓN 1: ULTRA CONSERVADORA
# Mayor generalización, menos overfit
# ==========================================
params_ultra_conservative = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Learning muy bajo para aprendizaje gradual
    'learning_rate': 0.005,
    'n_estimators': 10000,

    # Árboles muy simples
    'max_depth': 3,
    'min_child_weight': 30,
    'max_leaves': 15,

    # Sampling muy agresivo para diversidad
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'colsample_bynode': 0.6,

    # Balance de clases
    'scale_pos_weight': optimal_scale,

    # Regularización muy fuerte
    'reg_alpha': 2.0,      # L1
    'reg_lambda': 5.0,     # L2
    'gamma': 3.0,          # Min loss reduction
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 2: CONSERVADORA BALANCEADA
# Buen equilibrio generalización/performance
# ==========================================
params_conservative_balanced = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 7500,

    'max_depth': 4,
    'min_child_weight': 20,
    'max_leaves': 20,

    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.75,
    'colsample_bynode': 0.8,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.0,
    'reg_lambda': 3.0,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 3: MODERADA
# Tu configuración actual mejorada
# ==========================================
params_moderate = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,
    'max_leaves': 31,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 0.5,
    'reg_lambda': 2.5,
    'gamma': 1.5,
    'max_delta_step': 2,
}

# ==========================================
# CONFIGURACIÓN 4: AGRESIVA PARA RECALL
# Maximiza detección de defaults
# ==========================================
params_high_recall = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.015,
    'n_estimators': 5000,

    # Árboles más profundos para capturar patrones complejos
    'max_depth': 6,
    'min_child_weight': 8,
    'max_leaves': 50,

    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.85,
    'colsample_bynode': 0.9,

    # Más peso a clase positiva
    'scale_pos_weight': optimal_scale * 1.3,

    # Regularización más suave
    'reg_alpha': 0.3,
    'reg_lambda': 1.5,
    'gamma': 0.5,
    'max_delta_step': 3,
}

# ==========================================
# CONFIGURACIÓN 5: AGRESIVA PARA PRECISION
# Minimiza falsos positivos
# ==========================================
params_high_precision = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 4309,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    # Menos peso a positivos para ser más selectivo
    'scale_pos_weight': optimal_scale * 0.8,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 6: DART (Dropout Trees)
# Previene overfit con dropout en árboles
# ==========================================
params_dart = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',
    'booster': 'dart',  # Usa DART en lugar de gbtree

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # DART específicos
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'rate_drop': 0.1,
    'skip_drop': 0.5,

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 7: FAST & LIGHT
# Rápido para iteraciones, menor memoria
# ==========================================
params_fast = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Más rápido pero efectivo
    'learning_rate': 0.02,
    'n_estimators': 3000,

    'max_depth': 4,
    'min_child_weight': 20,
    'max_leaves': 15,
    'max_bin': 128,  # Reduce bins para velocidad

    'subsample': 0.7,
    'colsample_bytree': 0.7,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.0,
    'reg_lambda': 2.0,
    'gamma': 1.5,
}

# ==========================================
# CONFIGURACIÓN 8: DEEP TREES
# Árboles profundos con fuerte regularización
# ==========================================
params_deep = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 7000,

    # Muy profundos
    'max_depth': 8,
    'min_child_weight': 10,
    'max_leaves': 100,

    # Sampling fuerte para compensar profundidad
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'colsample_bynode': 0.6,

    'scale_pos_weight': optimal_scale,

    # Regularización muy fuerte
    'reg_alpha': 3.0,
    'reg_lambda': 5.0,
    'gamma': 4.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 9: MONOTONE CONSTRAINTS
# Fuerza relaciones lógicas en features
# ==========================================
params_monotone = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # Monotone constraints (ajustar según tus features)
    # +1: a mayor valor, mayor probabilidad de default
    # -1: a mayor valor, menor probabilidad de default
    # 0: sin restricción
    # Ejemplo para primeras 10 features (ajustar según tu caso)
    'monotone_constraints': '(0,0,0,0,0,1,1,-1,1,0)',  # Ejemplo

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 10: INTERACTION CONSTRAINTS
# Limita qué features pueden interactuar
# ==========================================
params_interaction = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # Permite interacciones solo dentro de grupos
    # Ejemplo: [[0,1,2], [3,4,5]] significa que features 0,1,2 pueden interactuar entre sí
    # y 3,4,5 entre sí, pero no entre grupos
    # 'interaction_constraints': [[0,1,2,3], [4,5,6,7], [8,9,10]],  # Ajustar a tus features

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 11: TWO-STAGE ENSEMBLE
# Entrena dos modelos complementarios
# ==========================================
params_ensemble_stage1 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Enfocado en generalización
    'learning_rate': 0.008,
    'n_estimators': 6000,

    'max_depth': 4,
    'min_child_weight': 25,

    'subsample': 0.7,
    'colsample_bytree': 0.7,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.5,
    'reg_lambda': 3.0,
    'gamma': 2.0,
}

params_ensemble_stage2 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Enfocado en capturar residuos
    'learning_rate': 0.012,
    'n_estimators': 4000,

    'max_depth': 6,
    'min_child_weight': 10,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale * 1.2,

    'reg_alpha': 0.3,
    'reg_lambda': 1.5,
    'gamma': 0.8,
}

# =========================================
# BASADO EN EL MEJOR AUC 5
# =========================================

params_best_auc = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,  # Ligeramente menor que 0.012
    'n_estimators': 5000,

    'max_depth': 5,
    'min_child_weight': 22,  # Entre 20 y 25
    'max_leaves': 28,

    'subsample': 0.75,
    'colsample_bytree': 0.76,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.85,  # Entre 0.8 y 1.0

    'reg_alpha': 0.7,
    'reg_lambda': 2.5,
    'gamma': 1.8,
    'max_delta_step': 1,
}

# ==========================================
# RESUMEN DE CONFIGURACIONES
# ==========================================
configs = {
    '1_ultra_conservative': params_ultra_conservative,
    '2_conservative_balanced': params_conservative_balanced,
    '3_moderate': params_moderate,
    '4_high_recall': params_high_recall,
    '5_high_precision': params_high_precision,
    #'6_dart': params_dart,         #Tarda como 3 horas en entrenar con gpu
    '7_fast': params_fast,
    '8_deep': params_deep,
    '9_monotone': params_monotone,
    '10_interaction': params_interaction,
    '11_ensemble_s1': params_ensemble_stage1,
    '11_ensemble_s2': params_ensemble_stage2,
    '12_best_auc': params_best_auc,
}

# ==========================================
# FUNCIÓN PARA ENTRENAR Y COMPARAR
# ==========================================
def train_and_evaluate(config_name, params, X_train, y_train, X_val, y_val):
    """
    Entrena un modelo con la configuración dada y evalúa
    """
    print(f"\n{'='*70}")
    print(f"Entrenando: {config_name}")
    print(f"{'='*70}")

    # Preparar parámetros
    train_params = params.copy()

    # Extraer n_estimators y early_stopping_rounds
    n_estimators = train_params.pop('n_estimators', 5000)
    early_stopping = min(500, n_estimators // 10)

    # Crear modelo
    model = XGBClassifier(
        **train_params,
        n_estimators=n_estimators,
        n_jobs=-1,
        random_state=42,
        enable_categorical=True, #added here too
        early_stopping_rounds=early_stopping
    )

    # Entrenar
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=200
    )

    # Evaluar
    from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix

    y_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    auc = roc_auc_score(y_val, y_proba)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    fpr = fp / (fp + tn)

    print(f"\n{'='*70}")
    print(f"RESULTADOS: {config_name}")
    print(f"{'='*70}")
    print(f"AUC:       {auc:.5f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"FPR:       {fpr:.4f}")
    print(f"Best iteration: {model.best_iteration}")

    return {
        'config': config_name,
        'model': model,
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'fpr': fpr,
        'best_iteration': model.best_iteration
    }

In [151]:
results = {}

for i, j in configs.items():
    results[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 1_ultra_conservative
[0]	validation_0-auc:0.71114
[200]	validation_0-auc:0.73489
[400]	validation_0-auc:0.74362
[600]	validation_0-auc:0.75167
[800]	validation_0-auc:0.75761
[1000]	validation_0-auc:0.76160
[1200]	validation_0-auc:0.76443
[1400]	validation_0-auc:0.76680
[1600]	validation_0-auc:0.76865
[1800]	validation_0-auc:0.77023
[2000]	validation_0-auc:0.77151
[2200]	validation_0-auc:0.77270
[2400]	validation_0-auc:0.77368
[2600]	validation_0-auc:0.77456
[2800]	validation_0-auc:0.77532
[3000]	validation_0-auc:0.77595
[3200]	validation_0-auc:0.77654
[3400]	validation_0-auc:0.77698
[3600]	validation_0-auc:0.77739
[3800]	validation_0-auc:0.77781
[4000]	validation_0-auc:0.77818
[4200]	validation_0-auc:0.77855
[4400]	validation_0-auc:0.77887
[4600]	validation_0-auc:0.77920
[4800]	validation_0-auc:0.77942
[5000]	validation_0-auc:0.77973
[5200]	validation_0-auc:0.77993
[5400]	validation_0-auc:0.78020
[5600]	validation_0-auc:0.78037
[5800]	validation_0-auc:0.78046
[6000]	valida

In [152]:
for key, d in results.items():
    print(key)
    print({k: d[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

1_ultra_conservative
{'auc': 0.78221111990238, 'precision': 0.7122383985441311, 'recall': 0.7169681703686741, 'fpr': 0.28967254408060455, 'best_iteration': 9817}
2_conservative_balanced
{'auc': 0.7833515611313256, 'precision': 0.7122106218792555, 'recall': 0.7185711014426379, 'fpr': 0.2903595145408747, 'best_iteration': 4711}
3_moderate
{'auc': 0.7835073237259546, 'precision': 0.7109498979823169, 'recall': 0.7181131211357912, 'fpr': 0.2919624456148386, 'best_iteration': 3508}
4_high_recall
{'auc': 0.7834621496895052, 'precision': 0.6844951923076923, 'recall': 0.7824593542477674, 'fpr': 0.3606594916418594, 'best_iteration': 1612}
5_high_precision
{'auc': 0.7830041956009035, 'precision': 0.7323943661971831, 'recall': 0.654911838790932, 'fpr': 0.23929471032745592, 'best_iteration': 2420}
7_fast
{'auc': 0.7836532806968852, 'precision': 0.7125368397188846, 'recall': 0.719716052209755, 'fpr': 0.2903595145408747, 'best_iteration': 1644}
8_deep
{'auc': 0.7826066483492023, 'precision': 0.711045

In [153]:
# ==========================================
# MICRO-OPTIMIZACIÓN DE 5_HIGH_PRECISION
# Exploración fina alrededor del mejor modelo
# ==========================================

# BASE DE REFERENCIA (5_high_precision - AUC: 0.78817)
# 'learning_rate': 0.012
# 'scale_pos_weight': optimal_scale * 0.80
# 'min_child_weight': 25
# 'max_leaves': 25
# 'reg_alpha': 0.8, 'reg_lambda': 2.5, 'gamma': 2.0

# ==========================================
# GRUPO 1: AJUSTE DE SCALE_POS_WEIGHT
# El hiperparámetro MÁS crítico según resultados
# ==========================================

# Config 13: Scale más bajo (más conservador)
params_13_scale_low = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.76,  # Entre 0.76-0.80

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 14: Scale ligeramente más alto
params_14_scale_med = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.82,  # Entre 0.80-0.85

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 2: AJUSTE DE MIN_CHILD_WEIGHT
# Segundo parámetro más sensible
# ==========================================

# Config 15: Más restrictivo (menos overfit)
params_15_mcw_high = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 28,  # Más alto que 25
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 16: Menos restrictivo (más capacidad)
params_16_mcw_low = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 22,  # Más bajo que 25
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 3: AJUSTE DE REGULARIZACIÓN (L1, L2, GAMMA)
# Balance entre los 3 tipos de regularización
# ==========================================

# Config 17: Regularización más suave
params_17_reg_soft = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.6,   # Menos L1
    'reg_lambda': 2.2,  # Menos L2
    'gamma': 1.7,       # Menos gamma
    'max_delta_step': 1,
}

# Config 18: Regularización más fuerte
params_18_reg_strong = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.0,   # Más L1
    'reg_lambda': 2.8,  # Más L2
    'gamma': 2.3,       # Más gamma
    'max_delta_step': 1,
}

# Config 19: Balance L1 vs L2 diferente
params_19_reg_balance = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.0,   # Más L1 (feature selection)
    'reg_lambda': 2.0,  # Menos L2 (suavidad)
    'gamma': 2.2,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 4: LEARNING RATE + MAX_LEAVES
# Combinaciones ajustadas
# ==========================================

# Config 20: Learning rate más lento, más árboles simples
params_20_slow_simple = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,  # Más lento
    'n_estimators': 7200,    # Más iteraciones

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 22,  # Más simple

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 21: Learning rate más rápido, árboles ligeramente más complejos
params_21_fast_complex = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.013,  # Más rápido
    'n_estimators': 5500,    # Menos iteraciones

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 28,  # Más complejo

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 5: SAMPLING VARIATIONS
# Ajustes en subsample y colsample_*
# ==========================================

# Config 22: Más sampling (más diversidad)
params_22_high_sample = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.78,           # Más alto
    'colsample_bytree': 0.78,    # Más alto
    'colsample_bylevel': 0.82,   # Más alto
    'colsample_bynode': 0.87,    # Más alto

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 23: Menos sampling (más conservador)
params_23_low_sample = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.72,           # Más bajo
    'colsample_bytree': 0.72,    # Más bajo
    'colsample_bylevel': 0.78,   # Más bajo
    'colsample_bynode': 0.83,    # Más bajo

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 6: COMBINACIONES ÓPTIMAS
# Mejores combinaciones basadas en intuición
# ==========================================

# Config 24: Combinación conservadora óptima
params_24_ultra_optimal = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 5,
    'min_child_weight': 26,  # Ligeramente más restrictivo
    'max_leaves': 24,        # Ligeramente más simple

    'subsample': 0.76,
    'colsample_bytree': 0.76,
    'colsample_bylevel': 0.81,
    'colsample_bynode': 0.86,

    'scale_pos_weight': optimal_scale * 0.78,  # Entre 0.76 y 0.80

    'reg_alpha': 0.85,  # Entre 0.8 y 0.9
    'reg_lambda': 2.6,  # Entre 2.5 y 2.7
    'gamma': 2.1,       # Entre 2.0 y 2.2
    'max_delta_step': 1,
}

# Config 25: Combinación agresiva óptima
params_25_balanced_optimal = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.0125,  # Entre 0.012 y 0.013
    'n_estimators': 5800,

    'max_depth': 5,
    'min_child_weight': 24,  # Ligeramente menos restrictivo
    'max_leaves': 26,        # Ligeramente más complejo

    'subsample': 0.77,
    'colsample_bytree': 0.77,
    'colsample_bylevel': 0.81,
    'colsample_bynode': 0.86,

    'scale_pos_weight': optimal_scale * 0.81,  # Entre 0.80 y 0.82

    'reg_alpha': 0.75,  # Menos L1
    'reg_lambda': 2.4,  # Menos L2
    'gamma': 1.9,       # Menos gamma
    'max_delta_step': 1,
}

# ==========================================
# DICCIONARIO DE TODAS LAS CONFIGS
# ==========================================
all_fine_tuned_configs = {
    '13_scale_low': params_13_scale_low,
    '14_scale_med': params_14_scale_med,
    '15_mcw_high': params_15_mcw_high,
    '16_mcw_low': params_16_mcw_low,
    '17_reg_soft': params_17_reg_soft,
    '18_reg_strong': params_18_reg_strong,
    '19_reg_balance': params_19_reg_balance,
    '20_slow_simple': params_20_slow_simple,
    '21_fast_complex': params_21_fast_complex,
    '22_high_sample': params_22_high_sample,
    '23_low_sample': params_23_low_sample,
    '24_ultra_optimal': params_24_ultra_optimal,
    '25_balanced_optimal': params_25_balanced_optimal,
}



In [154]:
results_tuned = {}

for i, j in all_fine_tuned_configs.items():
    results_tuned[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 13_scale_low
[0]	validation_0-auc:0.72014
[200]	validation_0-auc:0.75971
[400]	validation_0-auc:0.77112
[600]	validation_0-auc:0.77624
[800]	validation_0-auc:0.77895
[1000]	validation_0-auc:0.78085
[1200]	validation_0-auc:0.78197
[1400]	validation_0-auc:0.78266
[1600]	validation_0-auc:0.78292
[1800]	validation_0-auc:0.78329
[2000]	validation_0-auc:0.78346
[2200]	validation_0-auc:0.78365
[2400]	validation_0-auc:0.78381
[2600]	validation_0-auc:0.78352
[2800]	validation_0-auc:0.78353
[2814]	validation_0-auc:0.78348

RESULTADOS: 13_scale_low
AUC:       0.78386
Precision: 0.7366
Recall:    0.6423
FPR:       0.2297
Best iteration: 2314

Entrenando: 14_scale_med
[0]	validation_0-auc:0.71899
[200]	validation_0-auc:0.75986
[400]	validation_0-auc:0.77143
[600]	validation_0-auc:0.77621
[800]	validation_0-auc:0.77895
[1000]	validation_0-auc:0.78043
[1200]	validation_0-auc:0.78128
[1400]	validation_0-auc:0.78208
[1600]	validation_0-auc:0.78258
[1800]	validation_0-auc:0.78312
[2000]	val

In [155]:
for i, j in results_tuned.items():
    print(i)
    print({k: j[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

13_scale_low
{'auc': 0.7838631315313254, 'precision': 0.7366071428571429, 'recall': 0.6423173803526449, 'fpr': 0.229677123883673, 'best_iteration': 2314}
14_scale_med
{'auc': 0.7836176238834371, 'precision': 0.7314184933232553, 'recall': 0.6647584153881383, 'fpr': 0.24410350354934737, 'best_iteration': 2961}
15_mcw_high
{'auc': 0.7839755553666676, 'precision': 0.73559670781893, 'recall': 0.654911838790932, 'fpr': 0.23540187771925808, 'best_iteration': 2154}
16_mcw_low
{'auc': 0.7834370326106204, 'precision': 0.7353016688061618, 'recall': 0.6558277994046257, 'fpr': 0.23608884817952827, 'best_iteration': 2143}
17_reg_soft
{'auc': 0.7830423693658892, 'precision': 0.7318263549961469, 'recall': 0.6523929471032746, 'fpr': 0.23906572017403252, 'best_iteration': 2313}
18_reg_strong
{'auc': 0.7834753636850771, 'precision': 0.7333333333333333, 'recall': 0.6574307304785895, 'fpr': 0.23906572017403252, 'best_iteration': 2462}
19_reg_balance
{'auc': 0.784019287399632, 'precision': 0.734714760808390

In [156]:
xgb_model = XGBClassifier(**params_best_auc, n_jobs=-1, random_state=42, enable_categorical=True)
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.71879
[1]	validation_0-auc:0.73205
[2]	validation_0-auc:0.73597
[3]	validation_0-auc:0.73628
[4]	validation_0-auc:0.73777
[5]	validation_0-auc:0.73907
[6]	validation_0-auc:0.73949
[7]	validation_0-auc:0.73953
[8]	validation_0-auc:0.73983
[9]	validation_0-auc:0.73965
[10]	validation_0-auc:0.73937
[11]	validation_0-auc:0.73977
[12]	validation_0-auc:0.73969
[13]	validation_0-auc:0.74027
[14]	validation_0-auc:0.74049
[15]	validation_0-auc:0.74074
[16]	validation_0-auc:0.74089
[17]	validation_0-auc:0.74102
[18]	validation_0-auc:0.74151
[19]	validation_0-auc:0.74151
[20]	validation_0-auc:0.74157
[21]	validation_0-auc:0.74168
[22]	validation_0-auc:0.74158
[23]	validation_0-auc:0.74177
[24]	validation_0-auc:0.74212
[25]	validation_0-auc:0.74242
[26]	validation_0-auc:0.74254
[27]	validation_0-auc:0.74249
[28]	validation_0-auc:0.74250
[29]	validation_0-auc:0.74262
[30]	validation_0-auc:0.74264
[31]	validation_0-auc:0.74278
[32]	validation_0-auc:0.74298
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.85,
              colsample_bytree=0.76, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=1.8, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=5,
              max_leaves=28, min_child_weight=22, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [157]:
#optimal f1 threshold
from sklearn.metrics import f1_score
import numpy as np

y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1): {optimal_threshold:.3f}")

Optimal Threshold (F1): 0.344


In [158]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve

y_val_pred = xgb_model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_val_pred)

adj_threshold = optimal_threshold

y_val_pred = (y_val_pred >= adj_threshold).astype(int)

report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

FPR: 0.4479
Precision (PPV): 0.6548
AUC validación: 0.78288
              precision    recall  f1-score   support

           0       0.79      0.55      0.65      4367
           1       0.65      0.85      0.74      4367

    accuracy                           0.70      8734
   macro avg       0.72      0.70      0.69      8734
weighted avg       0.72      0.70      0.69      8734



In [159]:
# ==========================================
# CONFIGURACIONES XGBOOST PARA 363 FEATURES
# Optimizadas para dataset con feature engineering avanzado
# ==========================================

# IMPORTANTE: Con 363 features (vs ~120 originales), necesitas:
# 1. Más regularización (para evitar overfit con tantas features)
# 2. Más colsample (sampling de features más agresivo)
# 3. Árboles potencialmente más profundos (para capturar interacciones)

# ==========================================
# CONFIG 26: HIGH PRECISION CON MUCHAS FEATURES
# Adaptación del mejor modelo (5_high_precision) para 363 features
# ==========================================
params_26_hp_many_features = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,  # +1 vs original (más features = más profundidad OK)
    'min_child_weight': 30,  # +5 (más conservador con muchas features)
    'max_leaves': 30,  # +5

    # ⚡ CRÍTICO: Sampling más agresivo con 363 features
    'subsample': 0.70,  # -0.05
    'colsample_bytree': 0.65,  # -0.10 (muy importante!)
    'colsample_bylevel': 0.70,  # -0.10
    'colsample_bynode': 0.75,  # -0.10

    'scale_pos_weight': optimal_scale * 0.80,

    # Regularización más fuerte
    'reg_alpha': 1.2,   # +0.4
    'reg_lambda': 3.5,  # +1.0
    'gamma': 2.5,       # +0.5
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 27: ULTRA REGULARIZADO PARA MUCHAS FEATURES
# Previene overfit agresivamente
# ==========================================
params_27_ultra_reg = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,
    'n_estimators': 7000,

    'max_depth': 5,
    'min_child_weight': 35,
    'max_leaves': 25,

    # Feature sampling MUY agresivo
    'subsample': 0.65,
    'colsample_bytree': 0.60,
    'colsample_bylevel': 0.65,
    'colsample_bynode': 0.70,

    'scale_pos_weight': optimal_scale * 0.80,

    # Regularización extrema
    'reg_alpha': 2.0,
    'reg_lambda': 5.0,
    'gamma': 3.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 28: DEEP TREES CON FEATURE SELECTION
# Árboles profundos + sampling fuerte = explora interacciones
# ==========================================
params_28_deep_selection = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 8000,

    # Profundo para capturar interacciones complejas
    'max_depth': 7,
    'min_child_weight': 25,
    'max_leaves': 50,

    # Sampling agresivo compensa profundidad
    'subsample': 0.65,
    'colsample_bytree': 0.55,  # Muy bajo: cada árbol ve ~200 features
    'colsample_bylevel': 0.60,
    'colsample_bynode': 0.65,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 2.5,  # L1 alto para feature selection
    'reg_lambda': 4.0,
    'gamma': 3.5,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 29: BALANCED DIVERSITY
# Balance entre exploración y explotación
# ==========================================
params_29_balanced_diverse = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 6,
    'min_child_weight': 28,
    'max_leaves': 35,

    'subsample': 0.68,
    'colsample_bytree': 0.62,
    'colsample_bylevel': 0.68,
    'colsample_bynode': 0.73,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.5,
    'reg_lambda': 3.8,
    'gamma': 2.7,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 30: FEATURE SAMPLING EXTREMO
# Cada árbol ve solo ~30% de features (ensemble diverso)
# ==========================================
params_30_extreme_sampling = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.013,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 28,

    # SAMPLING EXTREMO
    'subsample': 0.60,
    'colsample_bytree': 0.50,  # Solo 180 features por árbol
    'colsample_bylevel': 0.55,
    'colsample_bynode': 0.60,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.8,
    'reg_lambda': 3.0,
    'gamma': 2.2,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 31: L1 DOMINANT (Feature Selection)
# Alta regularización L1 para selección automática de features
# ==========================================
params_31_l1_dominant = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 30,
    'max_leaves': 32,

    'subsample': 0.70,
    'colsample_bytree': 0.65,
    'colsample_bylevel': 0.70,
    'colsample_bynode': 0.75,

    'scale_pos_weight': optimal_scale * 0.80,

    # L1 muy alto para feature selection
    'reg_alpha': 3.0,  # ⚡ Muy alto
    'reg_lambda': 2.0,  # L2 más bajo
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 32: ADAPTIVE LEARNING
# Learning rate más alto con más regularización
# ==========================================
params_32_adaptive = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.015,  # Más rápido
    'n_estimators': 5000,

    'max_depth': 5,
    'min_child_weight': 32,
    'max_leaves': 27,

    'subsample': 0.68,
    'colsample_bytree': 0.63,
    'colsample_bylevel': 0.68,
    'colsample_bynode': 0.73,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.3,
    'reg_lambda': 3.3,
    'gamma': 2.5,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 33: MODERATE DEPTH + HIGH REG
# Balance clásico pero adaptado
# ==========================================
params_33_moderate_highreg = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 6,
    'min_child_weight': 27,
    'max_leaves': 33,

    'subsample': 0.72,
    'colsample_bytree': 0.67,
    'colsample_bylevel': 0.72,
    'colsample_bynode': 0.77,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.4,
    'reg_lambda': 3.6,
    'gamma': 2.6,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 34: CONSERVATIVE MANY FEATURES
# Muy conservador específicamente para 363 features
# ==========================================
params_34_conservative_363 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.009,
    'n_estimators': 7500,

    'max_depth': 4,  # Shallow
    'min_child_weight': 40,  # Muy restrictivo
    'max_leaves': 20,

    'subsample': 0.65,
    'colsample_bytree': 0.58,
    'colsample_bylevel': 0.63,
    'colsample_bynode': 0.68,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 2.2,
    'reg_lambda': 4.5,
    'gamma': 3.2,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 35: FOCUS ON NEW FEATURES
# Optimizado para aprovechar las nuevas features avanzadas
# ==========================================
params_35_new_features_focus = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 26,
    'max_leaves': 35,

    # Permite más exploración de features
    'subsample': 0.72,
    'colsample_bytree': 0.68,
    'colsample_bylevel': 0.73,
    'colsample_bynode': 0.78,

    'scale_pos_weight': optimal_scale * 0.78,  # Ligeramente ajustado

    'reg_alpha': 1.3,
    'reg_lambda': 3.2,
    'gamma': 2.3,
    'max_delta_step': 1,
}

# ==========================================
# DICCIONARIO COMPLETO
# ==========================================
all_advanced_configs = {
    '26_hp_many_features': params_26_hp_many_features,
    '27_ultra_reg': params_27_ultra_reg,
    '28_deep_selection': params_28_deep_selection,
    '29_balanced_diverse': params_29_balanced_diverse,
    '30_extreme_sampling': params_30_extreme_sampling,
    '31_l1_dominant': params_31_l1_dominant,
    '32_adaptive': params_32_adaptive,
    '33_moderate_highreg': params_33_moderate_highreg,
    '34_conservative_363': params_34_conservative_363,
    '35_new_features_focus': params_35_new_features_focus,
}


In [160]:
results_advanced = {}

for i, j in all_advanced_configs.items():
    results_advanced[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 26_hp_many_features
[0]	validation_0-auc:0.72163
[200]	validation_0-auc:0.76056
[400]	validation_0-auc:0.77265
[600]	validation_0-auc:0.77753
[800]	validation_0-auc:0.77999
[1000]	validation_0-auc:0.78153
[1200]	validation_0-auc:0.78236
[1400]	validation_0-auc:0.78291
[1600]	validation_0-auc:0.78302
[1800]	validation_0-auc:0.78332
[2000]	validation_0-auc:0.78334
[2200]	validation_0-auc:0.78341
[2400]	validation_0-auc:0.78331
[2600]	validation_0-auc:0.78316
[2635]	validation_0-auc:0.78313

RESULTADOS: 26_hp_many_features
AUC:       0.78345
Precision: 0.7305
Recall:    0.6572
FPR:       0.2425
Best iteration: 2135

Entrenando: 27_ultra_reg
[0]	validation_0-auc:0.72395
[200]	validation_0-auc:0.75546
[400]	validation_0-auc:0.76767
[600]	validation_0-auc:0.77333
[800]	validation_0-auc:0.77656
[1000]	validation_0-auc:0.77866
[1200]	validation_0-auc:0.78000
[1400]	validation_0-auc:0.78107
[1600]	validation_0-auc:0.78186
[1800]	validation_0-auc:0.78243
[2000]	validation_0-auc:0.78

In [161]:
for i, j in results_advanced.items():
    print(i)
    print({k: j[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

26_hp_many_features
{'auc': 0.7834475199086934, 'precision': 0.7304657673708322, 'recall': 0.657201740325166, 'fpr': 0.24250057247538356, 'best_iteration': 2135}
27_ultra_reg
{'auc': 0.7838287856301364, 'precision': 0.7348872950819673, 'recall': 0.6569727501717426, 'fpr': 0.2370048087932219, 'best_iteration': 3637}
28_deep_selection
{'auc': 0.7833446132963524, 'precision': 0.7357087926172776, 'recall': 0.657201740325166, 'fpr': 0.23608884817952827, 'best_iteration': 2996}
29_balanced_diverse
{'auc': 0.7844465923596153, 'precision': 0.7363984674329502, 'recall': 0.6601786123196702, 'fpr': 0.2363178383329517, 'best_iteration': 2351}
30_extreme_sampling
{'auc': 0.7826719317797065, 'precision': 0.7335206949412366, 'recall': 0.6574307304785895, 'fpr': 0.23883673002060912, 'best_iteration': 2692}
31_l1_dominant
{'auc': 0.7842331758438303, 'precision': 0.735969387755102, 'recall': 0.6606365926265171, 'fpr': 0.2370048087932219, 'best_iteration': 2853}
32_adaptive
{'auc': 0.7831125556082426, 'p

In [162]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

cat_features = [
    col for col in X.columns
    if X[col].dtype == 'category'
]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
aucs = []

for fold, (tr, va) in enumerate(skf.split(X, y), 1):

    model = CatBoostClassifier(
        iterations=4000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        early_stopping_rounds=300,
        verbose=200,
    )

    model.fit(
        X.iloc[tr], y.iloc[tr],
        eval_set=(X.iloc[va], y.iloc[va]),
        cat_features=cat_features
    )

    preds = model.predict_proba(X.iloc[va])[:, 1]
    auc = roc_auc_score(y.iloc[va], preds)
    aucs.append(auc)

    print(f'Fold {fold} AUC: {auc:.5f}')

print(f'\nAUC medio: {np.mean(aucs):.5f} ± {np.std(aucs):.5f}')



0:	test: 0.7188045	best: 0.7188045 (0)	total: 94.3ms	remaining: 6m 17s
200:	test: 0.7663852	best: 0.7663852 (200)	total: 20.9s	remaining: 6m 35s
400:	test: 0.7727573	best: 0.7727644 (399)	total: 42s	remaining: 6m 17s
600:	test: 0.7755787	best: 0.7755787 (600)	total: 1m 3s	remaining: 5m 57s
800:	test: 0.7761829	best: 0.7762892 (788)	total: 1m 31s	remaining: 6m 6s
1000:	test: 0.7769026	best: 0.7769136 (998)	total: 1m 53s	remaining: 5m 39s
1200:	test: 0.7776731	best: 0.7776991 (1193)	total: 2m 15s	remaining: 5m 16s
1400:	test: 0.7782719	best: 0.7783014 (1399)	total: 2m 43s	remaining: 5m 3s
1600:	test: 0.7782065	best: 0.7783401 (1502)	total: 3m 6s	remaining: 4m 39s
1800:	test: 0.7781525	best: 0.7783401 (1502)	total: 3m 26s	remaining: 4m 12s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7783401009
bestIteration = 1502

Shrink model to first 1503 iterations.
Fold 1 AUC: 0.77834
0:	test: 0.7293140	best: 0.7293140 (0)	total: 103ms	remaining: 6m 52s
200:	test: 0.7730908	b

In [163]:
print(aucs)

[0.7783401008741739, 0.7879316263822456, 0.775207911995209, 0.7871001409545297, 0.7826928015028717]


[0.7816351982003422, 0.7900056715882002, 0.7815013529893292, 0.789682262278456, 0.7824631521772024]

In [167]:
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold
thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f'optimal threshold: {optimal_threshold}')

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0,0]:6d}   {cm[0,1]:6d}")
print(f"Real Default   {cm[1,0]:6d}   {cm[1,1]:6d}")

optimal threshold: 0.41111111111111104
FPR: 0.3529
Precision (PPV): 0.7147
AUC validación: 0.85827
              precision    recall  f1-score   support

           0       0.85      0.65      0.73      4367
           1       0.71      0.88      0.79      4367

    accuracy                           0.77      8734
   macro avg       0.78      0.77      0.76      8734
weighted avg       0.78      0.77      0.76      8734


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def      2826     1541
Real Default      507     3860


In [165]:

# Lista de tus tres diccionarios
all_test = [results, results_advanced, results_tuned]

# Consolidamos todos los experimentos en una sola lista de items
all_items = []
for d in all_test:
    all_items.extend(d.items())

# Buscamos el item que tiene el valor máximo en la clave 'auc'
best_exp, best_info = max(all_items, key=lambda x: x[1]['auc'])

print(f"Best Experiment: {best_exp}")
print(f"Best AUC: {best_info['auc']}")

Best Experiment: 33_moderate_highreg
Best AUC: 0.7847125240205007


In [166]:
xgb_model = XGBClassifier(**params_18_reg_strong, n_jobs=-1, random_state=42, enable_categorical=True)
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.72035
[1]	validation_0-auc:0.73126
[2]	validation_0-auc:0.73454
[3]	validation_0-auc:0.73644
[4]	validation_0-auc:0.73749
[5]	validation_0-auc:0.73867
[6]	validation_0-auc:0.73863
[7]	validation_0-auc:0.73911
[8]	validation_0-auc:0.73967
[9]	validation_0-auc:0.73976
[10]	validation_0-auc:0.73991
[11]	validation_0-auc:0.73966
[12]	validation_0-auc:0.74023
[13]	validation_0-auc:0.74027
[14]	validation_0-auc:0.74029
[15]	validation_0-auc:0.74040
[16]	validation_0-auc:0.74052
[17]	validation_0-auc:0.74072
[18]	validation_0-auc:0.74095
[19]	validation_0-auc:0.74111
[20]	validation_0-auc:0.74119
[21]	validation_0-auc:0.74136
[22]	validation_0-auc:0.74167
[23]	validation_0-auc:0.74193
[24]	validation_0-auc:0.74214
[25]	validation_0-auc:0.74221
[26]	validation_0-auc:0.74224
[27]	validation_0-auc:0.74238
[28]	validation_0-auc:0.74247
[29]	validation_0-auc:0.74287
[30]	validation_0-auc:0.74275
[31]	validation_0-auc:0.74286
[32]	validation_0-auc:0.74303
[33]	validation_0-au

KeyboardInterrupt: 

In [94]:
# Probabilidades (idéntico conceptualmente)
y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f'Optimal Threshold: {optimal_threshold}')
#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0,0]:6d}   {cm[0,1]:6d}")
print(f"Real Default   {cm[1,0]:6d}   {cm[1,1]:6d}")

FPR: 0.1007
Precision (PPV): 0.2856
AUC validación: 0.78619
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     46061
           1       0.29      0.42      0.34      4367

    accuracy                           0.86     50428
   macro avg       0.61      0.66      0.63     50428
weighted avg       0.89      0.86      0.87     50428


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def     41423     4638
Real Default     2513     1854


In [30]:
import optuna
from sklearn.metrics import roc_auc_score

def objective(trial):

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'hist',
        'device': 'cuda',
        'booster': 'gbtree',

        # Ranges
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 3000, 8000),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 10, 40),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'max_delta_step': trial.suggest_float('max_delta_step', 0, 10),

        'scale_pos_weight': trial.suggest_float('scale_pos_weight', optimal_scale * 0.7, optimal_scale * 1.3),

        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 5.0),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
    }

    # Training
    model = XGBClassifier(**params, n_jobs=-1, random_state=42, early_stopping_rounds=100, enable_categorical=True)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    # AUC
    preds = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, preds)

# Progress callback
def callback(study, trial):
    if trial.number % 10 == 0:
        print(f"Trial {trial.number}/150: Current={trial.value:.5f}, Best={study.best_value:.5f}")

# 50 tests
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150, callbacks=[callback] ,show_progress_bar=True)

best_params = study.best_params
print("Bests params:", best_params)

[I 2026-01-10 11:32:01,774] A new study created in memory with name: no-name-4676bb08-c5cd-462c-9121-34776134fb9b


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2026-01-10 11:32:14,426] Trial 0 finished with value: 0.7795670600548249 and parameters: {'learning_rate': 0.015489198397449363, 'n_estimators': 7226, 'max_depth': 3, 'min_child_weight': 27, 'subsample': 0.80264079272502, 'colsample_bytree': 0.802244337671741, 'max_delta_step': 3.277844056442698, 'scale_pos_weight': 1.2436502156410734, 'reg_alpha': 3.5640328046721494, 'reg_lambda': 1.2863426241884062, 'gamma': 4.578314725350239}. Best is trial 0 with value: 0.7795670600548249.
Trial 0/150: Current=0.77957, Best=0.77957
[I 2026-01-10 11:32:29,132] Trial 1 finished with value: 0.7793151455962128 and parameters: {'learning_rate': 0.007967887707963823, 'n_estimators': 4223, 'max_depth': 4, 'min_child_weight': 30, 'subsample': 0.8691220092980273, 'colsample_bytree': 0.7215596053951303, 'max_delta_step': 6.492930744409657, 'scale_pos_weight': 0.781660516385146, 'reg_alpha': 1.8688502084857301, 'reg_lambda': 1.7817171185230003, 'gamma': 2.2374284946613088}. Best is trial 0 with value: 0.77